In [ ]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.python import debug as tf_debug

import numpy as np
import cv2
import math
import os
import random
import time
import pprint
from datetime import datetime

from model import Model # tensorflow model define file
import input

In [ ]:
IMAGE_SIZE = 256
INPUT_SIZE = 240
DST_INPUT_SIZE = 128

NUM_CLASS = 21

In [ ]:
LOGDIR = 'log/data_%s/' % datetime.now().strftime("%Y-%m-%d_%H%M")
flags = tf.app.flags
FLAGS = flags.FLAGS
flags.DEFINE_string('trainDir', LOGDIR, 'DIrectory to put the training data')
flags.DEFINE_string('tfrecordFileName', "train_tf_file_%sx%s.tfrecords" % (DST_INPUT_SIZE, DST_INPUT_SIZE), 'File name of train tfrecord')
flags.DEFINE_integer('maxSteps', 200000, 'Number of steps to run trainer')
flags.DEFINE_integer('batchSize', 90, 'train data size of subset')
flags.DEFINE_integer('numExamplesPerEpochForTrain', 20000, 'input image num(include inflation)')
flags.DEFINE_integer('threadNum', 4, 'num of threads')
flags.DEFINE_float('learningLate', 1e-4, 'Initial learning rate')
flags.DEFINE_string('f', '', 'kernel') # エラー回避

In [ ]:
def _parse_function(exampleProto):
    features={
            'label': tf.FixedLenFeature((), tf.int64, default_value=0),
            'image': tf.FixedLenFeature((), tf.string, default_value="")
        }
    parsedFeatures = tf.parse_single_example(exampleProto, features)  # データ構造を解析
     
    return parsedFeatures["image"], parsedFeatures["label"]
 
# データ解析（２）
def read_image(argImages, argLabels):      
    images = tf.decode_raw(argImages, tf.uint8)
    images = tf.cast(images, tf.float32)
    images = tf.reshape(images, [DST_INPUT_SIZE, DST_INPUT_SIZE, 3])
    
    images = tf.image.resize_images(images, [IMAGE_SIZE, IMAGE_SIZE], method=tf.image.ResizeMethod.BICUBIC)
    
    ### 水増し
    # 切り取り
    cropsize = random.randint(INPUT_SIZE, INPUT_SIZE + (IMAGE_SIZE - INPUT_SIZE) / 2)
    framesize = INPUT_SIZE + (cropsize - INPUT_SIZE) * 2
    images = tf.image.resize_image_with_crop_or_pad(images, framesize, framesize)
    images = tf.random_crop(images, [cropsize, cropsize, 3])
    
    # 回転
    angle = random.randint(-15, 15)
    radian = angle * math.pi / 180
    images = tf.contrib.image.rotate(images, radian, interpolation="BILINEAR")
    
    # 左右反転
    images = tf.image.random_flip_left_right(images)
    
    # 輝度変化
    images = tf.image.random_brightness(images, max_delta=0.2)
    
    # コントラスト変化
    images = tf.image.random_contrast(images, lower=0.6, upper=1.4)
    images = tf.image.random_hue(images, max_delta=0.04)
    images = tf.image.random_saturation(images, lower=0.6, upper=1.4)
    
    # サイズもとに戻す
    images = tf.image.resize_images(images, [DST_INPUT_SIZE, DST_INPUT_SIZE], method=tf.image.ResizeMethod.BICUBIC)
    
    images = images / 255
    images = tf.reshape(images, [DST_INPUT_SIZE, DST_INPUT_SIZE, 3])
    
    labels = tf.cast(argLabels, tf.int32)
    labels = tf.one_hot(labels, NUM_CLASS)
     
    return images, labels

In [ ]:
 with tf.Graph().as_default():
        
    threshold = 1e-10 # これ以下の誤差であれば終了
    bestLossResult = 0.01
    bestResult = []
    bestStep = None
    bestSteps = []
    bestSession = None
        
    # 入力層
    filenames = tf.placeholder(tf.string, shape=[None])
    dataset = tf.data.TFRecordDataset(filenames)
    dataset = dataset.map(_parse_function, FLAGS.threadNum)  # レコードを解析し、テンソルに変換
    dataset = dataset.map(read_image, FLAGS.threadNum)  # データの形式、形状を変更
    dataset = dataset.shuffle(FLAGS.numExamplesPerEpochForTrain)
    dataset = dataset.batch(FLAGS.batchSize)  # 連続するレコードをバッチに結合
    dataset = dataset.repeat(-1)  # 無限に繰り返す
    iterator = tf.data.Iterator.from_structure(dataset.output_types, dataset.output_shapes)  # イテレータを作成
    images, labels = iterator.get_next()  # イテレータの次の要素を取得
    tf.summary.image('image', images, max_outputs = 100)

    initOp = iterator.make_initializer(dataset)  # イテレータを初期化
        
    model = Model(images, labels, DST_INPUT_SIZE, NUM_CLASS, FLAGS.batchSize, trainable=True)
        
    globalStep = tf.train.get_or_create_global_step()
    optimizer = tf.train.AdamOptimizer(FLAGS.learningLate)
    extraUpdataOps = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(extraUpdataOps):
        trainOp = optimizer.minimize(model.loss, global_step=globalStep)

    tf.add_to_collection('train_op', trainOp)

    with tf.Session() as session:
        init = tf.global_variables_initializer()
        session.run(init)  # 変数の初期化処理
        session.run(initOp, feed_dict={filenames: ["train_tf_file_%sx%s.tfrecords" % (DST_INPUT_SIZE, DST_INPUT_SIZE)]})  # データの初期化
        
        saver = tf.train.Saver(max_to_keep = 0)
        session.run(tf.global_variables_initializer())

        summaryOp = tf.summary.merge_all()
        summaryWriter = tf.summary.FileWriter(FLAGS.trainDir, session.graph)

        for i in range(FLAGS.maxSteps):
            startTime = time.time()
            _, lossResult, accuracyResult = session.run([trainOp, model.loss, model.accuracy], feed_dict={model.keepProb: 0.75, model.isTraining: True})
            step = session.run(globalStep)
            duration = time.time() - startTime

            if step % 10 == 0:
                numExamplesPerStep = FLAGS.batchSize
                examplesPerSec = numExamplesPerStep / duration
                secPerBatch = float(duration)
                formatStr = ('%s: step %d, loss = %.5f (%.1f examples/sec; %.3f sec/batch)')
                print(formatStr % (datetime.now(), step, lossResult, examplesPerSec, secPerBatch))
                print('accuracy result', accuracyResult)

            if step % 25 == 0 or step == 1:
                summaryStr = session.run(summaryOp, feed_dict={model.keepProb: 1.0, model.isTraining: True})
                summaryWriter.add_summary(summaryStr, step)

            if step % 1000 == 0 or step == FLAGS.maxSteps or lossResult < threshold:
                # 1000回ごとの最良を保存
                if bestSession is not None:
                    checkpointPath = os.path.join(FLAGS.trainDir, 'model.ckpt')
                    saver.save(bestSession, checkpointPath, global_step = bestStep)
                    bestResult.append(bestLossResult)
                    bestSteps.append(bestStep)
                    bestLossResult = 0.01
                    bestStep = None
                    bestSession = None
                else:
                    checkpointPath = os.path.join(FLAGS.trainDir, 'model.ckpt')
                    saver.save(session, checkpointPath, global_step = step)

            if lossResult < bestLossResult:
                bestLossResult = lossResult
                bestStep = step
                bestSession = session

            if lossResult < threshold:
                print('loss is zero')
                break